CODIFICAR PARA ESCALA DE GRISES

In [ ]:
import numpy as np
from PIL import Image
import struct


def escribeIm(x, nombre):
    """
    x: numpy array de tipo uint8 con la imagen en escala de grises [0..255].
    nombre: nombre del fichero binario donde se almacenará la imagen codificada.
    """
    # Verificar que x es uint8
    if x.dtype != np.uint8:
        raise ValueError("La imagen debe ser uint8.")

    # Obtenemos filas y columnas
    f, c = x.shape

    with open(nombre, "wb") as fid:
        # 1) Escribimos el número de filas (16 bits)
        # Utilizamos struct para empacar en formato little-endian ('<')
        # 'H' (unsigned short) => 16 bits sin signo
        fid.write(struct.pack("<H", f))

        # 2) Escribimos el número de columnas (16 bits)
        fid.write(struct.pack("<H", c))

        # 3) Escribimos los píxeles en orden de columnas
        # En MATLAB, fread/fwrite con matrices lo hace por columnas,
        # mientras que en NumPy (row-major) x.flatten() sería por filas.
        # Para imitar al 100% la escritura por COLUMNAS, podemos trasponer x y luego hacer un flatten:
        x_col_major = x.T.flatten()
        # x.T -> invierte filas y columnas
        # flatten() -> saca un vector en orden row-major, pero de la imagen traspuesta.

        # Ahora escribimos esos bytes:
        x_col_major.tofile(fid)  # Equivalente a un fwrite de todos los datos.


def ejemplo_escribeIm():
    # Cargar i1.png como escala de grises
    # (Podrías tener la imagen en color, y convertirla a gris)
    im_rgb = Image.open("../data/i1.png").convert("L")  # "L" => 8 bits (grayscale)
    x = np.array(im_rgb, dtype=np.uint8)  # Convertimos a numpy array

    # Codificar la imagen a un fichero binario
    escribeIm(x, "1i.bin")  # El nombre "i1i.bin" es el "fichero codificado"

    print("Fichero 'i1i.bin' generado con cabecera + datos de imagen.")


ejemplo_escribeIm()

# numero de filas maximo que puede tener la imagen a codificar
max_filas = 65535

Fichero 'i1i.bin' generado con cabecera + datos de imagen.


DECODIFICAR EN ESCALA DE GRISES

In [8]:
def leeIm(nombre):
    """
    nombre: nombre del fichero binario con la secuencia de bits a decodificar.
    Retorna la imagen decodificada (numpy array uint8).
    """
    with open(nombre, "rb") as fid:
        # 1) Leer f y c como uint16
        contenido = fid.read(2)  # leer 2 bytes -> f
        f = struct.unpack("<H", contenido)[0]

        contenido = fid.read(2)  # leer 2 bytes -> c
        c = struct.unpack("<H", contenido)[0]

        # 2) Leer f*c bytes (uint8)
        num_pix = f * c
        # leemos en binario
        datos = fid.read(num_pix)
        # Convertir a array numpy
        arr = np.frombuffer(datos, dtype=np.uint8)

        # 3) Reconstruir la matriz en orden de columnas.
        # En MATLAB, fread(...,[f c],'uint8') rellena la matriz por columnas.
        # Para imitarlo:
        #   - arr es un vector de tamaño f*c
        #   - lo interpretamos como c columnas y f filas en row-major
        #   - luego lo trasponemos
        arr_2d_col = arr.reshape((c, f)).T  # Ahora es (f, c)

        # arr_2d_col es la imagen final
        return arr_2d_col


def ejemplo_leeIm():
    y = leeIm("i1i.bin")
    print("Dimensiones imagen decodificada:", y.shape)

    # Mostrar
    import matplotlib.pyplot as plt

    plt.imshow(y, cmap="gray")
    plt.title("Imagen decodificada (escala de grises)")
    plt.axis("off")
    plt.show()

    return y

CODIFICADOR/DECODFICADOR PARA IMÁGENES RGB

In [9]:
def rgb_to_y(im_rgb):
    """
    Convierte array RGB (uint8) a su luminancia Y (uint8).

    Podrías usar una fórmula como:
       Y = 0.299 R + 0.587 G + 0.114 B
    redondeando al uint8.
    """
    # im_rgb: (filas, cols, 3) en uint8
    R = im_rgb[:, :, 0].astype(np.float32)
    G = im_rgb[:, :, 1].astype(np.float32)
    B = im_rgb[:, :, 2].astype(np.float32)

    Y_float = 0.299 * R + 0.587 * G + 0.114 * B
    Y = np.clip(Y_float, 0, 255).astype(np.uint8)

    return Y


def escribeRGB(nombre_im, nombre_sb, color=1):
    """
    nombre_im: fichero de imagen en disco (por ej. 'i1.png')
    nombre_sb: nombre del fichero binario de salida
    color: 1 => codifica R, G, B (24 bpp); 0 => codifica solo la componente Y (8 bpp).
    """
    # 1) Leemos la imagen con Pillow
    im = Image.open(nombre_im).convert("RGB")  # Aseguramos que está en RGB
    arr = np.array(im, dtype=np.uint8)  # (f, c, 3)

    f, c, _ = arr.shape

    with open(nombre_sb, "wb") as fid:
        # Escribimos un indicador en la cabecera: color o no
        # Para simplificar, 1 byte con color (1 ó 0)
        fid.write(struct.pack("B", color))

        # Escribimos f y c (uint16)
        fid.write(struct.pack("<H", f))
        fid.write(struct.pack("<H", c))

        if color == 1:
            # Codificamos R, G y B => 3 canales => 24 bits/píxel
            # (f,c,3). Queremos escribir cada canal en orden R->G->B
            # y por columnas.

            # Separamos
            R = arr[:, :, 0]
            G = arr[:, :, 1]
            B = arr[:, :, 2]

            # Usaremos la misma lógica de "column-major":
            R_flat_col = R.T.flatten()
            G_flat_col = G.T.flatten()
            B_flat_col = B.T.flatten()

            # Escribimos consecutivamente
            R_flat_col.tofile(fid)
            G_flat_col.tofile(fid)
            B_flat_col.tofile(fid)

        else:
            # color == 0 => Solo guardamos la componente Y (8 bpp)
            Y = rgb_to_y(arr)
            Y_flat_col = Y.T.flatten()
            Y_flat_col.tofile(fid)

In [10]:
def leeRGB(nombre_sb):
    with open(nombre_sb, "rb") as fid:
        # 1) Leemos color (1 byte)
        contenido = fid.read(1)
        color_flag = struct.unpack("B", contenido)[0]

        # 2) Leemos f y c (16 bits cada uno)
        contenido = fid.read(2)
        f = struct.unpack("<H", contenido)[0]
        contenido = fid.read(2)
        c = struct.unpack("<H", contenido)[0]

        if color_flag == 1:
            # 3 canales
            num_pix = f * c
            # Leemos R, G, B por columnas
            R_data = fid.read(num_pix)
            G_data = fid.read(num_pix)
            B_data = fid.read(num_pix)

            R_arr = np.frombuffer(R_data, dtype=np.uint8).reshape((c, f)).T
            G_arr = np.frombuffer(G_data, dtype=np.uint8).reshape((c, f)).T
            B_arr = np.frombuffer(B_data, dtype=np.uint8).reshape((c, f)).T

            # Reconstruir la imagen
            out = np.stack([R_arr, G_arr, B_arr], axis=2)  # (f, c, 3)

        else:
            # Solo luminancia
            num_pix = f * c
            Y_data = fid.read(num_pix)
            Y_arr = np.frombuffer(Y_data, dtype=np.uint8).reshape((c, f)).T

            # Convertimos a (f,c,3) replicando la luminancia en R=G=B si queremos mostrarla en color
            # o la dejamos en 2D. Para imshow en escala de grises con matplotlib, nos basta 2D.
            # Sin embargo, la práctica dice que la imagen se mostrará en gama de grises.
            out = Y_arr  # 2D

    return out


def ejemplo_escribeLeeRGB():
    # Codifica i1.png de dos formas: (1) color, (2) gris

    # *** CODIFICACIÓN Y LECTURA EN COLOR ***
    escribeRGB("i1.png", "i1_color.bin", color=1)
    out_color = leeRGB("i1_color.bin")
    print("Forma out_color:", out_color.shape)  # (f, c, 3)

    # Mostramos
    import matplotlib.pyplot as plt

    plt.figure()
    plt.imshow(out_color)
    plt.title("Decodificado - RGB")
    plt.axis("off")

    # *** CODIFICACIÓN Y LECTURA EN ESCALA DE GRISES (solo Y) ***
    escribeRGB("i1.png", "i1_gray.bin", color=0)
    out_gray = leeRGB("i1_gray.bin")
    print("Forma out_gray:", out_gray.shape)  # (f, c)

    plt.figure()
    plt.imshow(out_gray, cmap="gray")
    plt.title("Decodificado - Y (Escala de grises)")
    plt.axis("off")

    plt.show()